## How to Remove Multicollinearity Using Python

- https://towardsdatascience.com/how-to-remove-multicollinearity-using-python-4da8d9d8abb2

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

### Data

In [2]:
df = pd.read_csv("../data/weatherAUS.csv")

In [3]:
print(df.shape)
display(df.head())

(145460, 23)


Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7       1007.1       8.0       NaN     16.9     21.8         No   
1       1010.6       1007.8       NaN       NaN     17.2     24.3         No   
2       1007.6       1008.7       NaN       2.0     21.0     23.2         No   
3       1017.6       1012.8       NaN       NaN     18.1     26.5         No   
4       1010.8       1006.0       7.0       8.0     17.8     29.7         No   

   RainTomorrow  
0            No  
1            No  
2            No  
3            No  
4            No  

[5 rows x 23 columns]

### 전처리

In [4]:
df = df[list(df.columns[2:])]
df = df.drop(['WindGustDir', 'WindDir9am', 'WindDir3pm'], axis=1)
df = df.dropna()
print(df.shape)
df.head()

(58090, 18)


MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
6049     17.9     35.2       0.0         12.0      12.3           48.0   
6050     18.4     28.9       0.0         14.8      13.0           37.0   
6052     19.4     37.6       0.0         10.8      10.6           46.0   
6053     21.9     38.4       0.0         11.4      12.2           31.0   
6054     24.2     41.0       0.0         11.2       8.4           35.0   

      WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  \
6049           6.0          20.0         20.0         13.0       1006.3   
6050          19.0          19.0         30.0          8.0       1012.9   
6052          30.0          15.0         42.0         22.0       1012.3   
6053           6.0           6.0         37.0         22.0       1012.7   
6054          17.0          13.0         19.0         15.0       1010.7   

      Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm RainToday RainTomorrow  
6049       1004.4       2.0       5.0     26.6     33.4        No           No  
6050       1012.1       1.0       1.0     20.3     27.0        No           No  
6052       1009.2       1.0       6.0     28.7     34.9        No           No  
6053       1009.1       1.0       5.0     29.1     35.6        No           No  
6054       1007.4       1.0       6.0     33.6     37.6        No           No

### VIF 계산

VIF는 변수에 다중공선성이 있는지 여부를 결정하는 숫자입니다.   
이 숫자는 또한 다른 변수와의 선형 의존성으로 인해 변수가 얼마나 부풀려졌는지를 나타냅니다.  

VIF 값은 1부터 시작하며 상한은 없습니다.   
숫자가 커지면 변수의 다중공선성이 크다는 것을 의미합니다.  

VIF를 계산하기 위해 각 변수에 대해 선형 회귀 프로세스를 수행하며, 여기서 해당 변수가 대상 변수가 됩니다.   
이 프로세스를 수행한 후 R 제곱을 계산합니다.   
마지막으로 이 공식을 사용하여 VIF 값을 계산합니다.

- VIF = 1 /(1-R^2)

statesmodels의 variance_inflation_factor를 사용해서 VIF를 사용할 수 있다.

In [5]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[list(df.columns[:-2])]
X.head()

MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
6049     17.9     35.2       0.0         12.0      12.3           48.0   
6050     18.4     28.9       0.0         14.8      13.0           37.0   
6052     19.4     37.6       0.0         10.8      10.6           46.0   
6053     21.9     38.4       0.0         11.4      12.2           31.0   
6054     24.2     41.0       0.0         11.2       8.4           35.0   

      WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  \
6049           6.0          20.0         20.0         13.0       1006.3   
6050          19.0          19.0         30.0          8.0       1012.9   
6052          30.0          15.0         42.0         22.0       1012.3   
6053           6.0           6.0         37.0         22.0       1012.7   
6054          17.0          13.0         19.0         15.0       1010.7   

      Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  
6049       1004.4       2.0       5.0     26.6     33.4  
6050       1012.1       1.0       1.0     20.3     27.0  
6052       1009.2       1.0       6.0     28.7     34.9  
6053       1009.1       1.0       5.0     29.1     35.6  
6054       1007.4       1.0       6.0     33.6     37.6

In [6]:
vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

VIF         Column
10  425849.003879    Pressure9am
11  424035.416584    Pressure3pm
15     673.638775        Temp3pm
1      608.073689        MaxTemp
14     208.616616        Temp9am
8       60.741577    Humidity9am
0       57.816330        MinTemp
9       47.893909    Humidity3pm
5       26.320937  WindGustSpeed
4       17.288336       Sunshine
7       13.476810   WindSpeed3pm
13       8.363979       Cloud3pm
6        7.776568   WindSpeed9am
12       7.402820       Cloud9am
3        7.093691    Evaporation
2        1.274984       Rainfall

- 대부분의 VIF 값이 5 이상이다. 심지어 압력 관련 변수는 400 이상이다.

### Remove multicollinearities

multicolliearity 제거 두방법
- 새로운 피처 만들기
- 피처 제거   

그런데 피처 제거는 초기에 추천하는 방법은 아니다.   
정보 손실이 발생하기 때문이다.   
따라서 피처 생성을 우선 실행해본다.  

위에서 피처 짝이 보인다.  
- ‘Temp9am’ with ‘Temp3pm’, ‘Pressure9am’ with ‘Pressure3pm’, ‘Cloud9am’ with ‘Cloud3pm’   

위를 바탕으로 새로운 피처를 생성할 수 있다.   
새로운 피처는 두 패어의 차(difference)이다.  
피처 생성 후에 안정적으로 제거할 수 있다.

In [7]:
df['TempDiff'] = df['Temp3pm'] - df['Temp9am']
df['HumidityDiff'] = df['Humidity3pm'] - df['Humidity9am']
df['CloudDiff'] = df['Cloud3pm'] - df['Cloud9am']
df['WindSpeedDiff'] = df['WindSpeed3pm'] - df['WindSpeed9am']
df['PressureDiff'] = df['Pressure3pm'] - df['Pressure9am']

X = df.drop(['Temp3pm', 'Temp9am', 'Humidity3pm', 'Humidity9am', 'Cloud3pm', 'Cloud9am', 'WindSpeed3pm', 'WindSpeed9am', 'Pressure3pm', 'Pressure9am', 'RainToday', 'RainTomorrow'], axis=1)

X.head()

MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
6049     17.9     35.2       0.0         12.0      12.3           48.0   
6050     18.4     28.9       0.0         14.8      13.0           37.0   
6052     19.4     37.6       0.0         10.8      10.6           46.0   
6053     21.9     38.4       0.0         11.4      12.2           31.0   
6054     24.2     41.0       0.0         11.2       8.4           35.0   

      TempDiff  HumidityDiff  CloudDiff  WindSpeedDiff  PressureDiff  
6049       6.8          -7.0        3.0           14.0          -1.9  
6050       6.7         -22.0        0.0            0.0          -0.8  
6052       6.2         -20.0        5.0          -15.0          -3.1  
6053       6.5         -15.0        4.0            0.0          -3.6  
6054       4.0          -4.0        5.0           -4.0          -3.3

다시 VIF를 계산해보자.

In [8]:
vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

VIF         Column
1   91.131365        MaxTemp
0   34.935952        MinTemp
6   14.507768       TempDiff
4    8.940550       Sunshine
5    6.924693  WindGustSpeed
3    5.858551    Evaporation
7    5.583406   HumidityDiff
10   3.786961   PressureDiff
9    1.388523  WindSpeedDiff
2    1.217792       Rainfall
8    1.082814      CloudDiff

- 여전히 높은 값이 있다.   

이 높은 피처를 제거해본다.

In [9]:
X = X.drop(['MaxTemp', 'MinTemp', 'TempDiff', 'Sunshine'], axis=1)

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

VIF         Column
2  4.552751  WindGustSpeed
1  3.372872    Evaporation
6  2.504805   PressureDiff
3  2.009856   HumidityDiff
5  1.286986  WindSpeedDiff
0  1.130127       Rainfall
4  1.024887      CloudDiff

- 이제 모든 값들이 5이하가 되었다.

### Build model

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [11]:
y = df["RainToday"]
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
print(encoder.classes_)
print(y_encoded)

['No' 'Yes']
[0 0 0 ... 0 0 0]


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y_encoded)

model = SVC()
model.fit(X.values, y_encoded)

print(model.score(X_test, y_test))

0.9881567169317634


### Interpretation using permutation feature importance

이론적으로 SVM 모델은 해석이 불가능합니다. 매개변수만 보고 결과를 해석할 수 없기 때문입니다. 하지만 다행히도 이 모델을 해석할 수 있는 몇 가지 방법이 있습니다. 우리가 사용할 수 있는 방법 중 하나는 순열 기능 중요도입니다.

순열 특징 중요도는 특징 값을 변경한 후 오류가 얼마나 증가하는지를 살펴봄으로써 특징의 중요성을 측정합니다. 해당 값의 변경으로 모델의 오류가 증가하면 해당 기능은 중요합니다.

이 방법을 구현하기 위해 scikit-learn 라이브러리의 permutation_importance라는 함수를 사용하여 특징 중요도를 계산할 수 있습니다. 이 결과를 바탕으로 특징 중요도를 시각화하기 위해 박스 플롯을 만들 것입니다.

In [13]:
# from sklearn.inspection import permutation_importance

# result = permutation_importance(model, X.values, y_encoded, n_repeats=10, random_state=42)

# perm_imp_idx = result.importances_mean.argsort()
# plt.boxplot(result.importances[perm_imp_idx].T, vert=False,
#             labels=X.columns[perm_imp_idx])
# plt.title('Feature Importance from Rain in Australia Dataset')
# plt.show()